In [1]:
import os
from pathlib import Path
import re
import pickle
import random

def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def save_obj(name, obj):
    existeDir = os.path.exists('obj')
    if not existeDir:
        os.makedirs('obj')
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('obj')
    if not existeDir:
        os.makedirs('obj')
    try:
        with open('obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None


In [3]:
import pandas as pd
arquivos = pd.read_csv('CRAM/genia.test.iob2', delimiter='\t', skip_blank_lines = False, header = None)
arquivos.columns=["token", "Label1", "Label2", "Label3", "Label4"]
print(arquivos.head())

def is_nan(x):
    return (x != x)

def getEntidades(arquivos):
    dicEntidades = {}
    listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

    tokens = arquivos['token']
    label1 = arquivos['Label1']
    label2 = arquivos['Label2']
    label3 = arquivos['Label3']
    label4 = arquivos['Label4']

    fraseCompleta=''
    i=0
    for token in tokens:
        #print(token)
        #print(is_nan(token))
        if token and not is_nan(token):
            fraseCompleta = fraseCompleta + ' ' + token
        else:
            dicEntidades[i] = [fraseCompleta, []]
            fraseCompleta = ""
            i=i+1    
        #if i>10:
        #    break

    #dicEntidades

    listaEntidades=[]
    isEnt = 0
    indices=[]
    for tag in listaTags:
        i=0
        j=0
        #print(tag)
        for token, l1 in zip(tokens, label1):
            #print(token)
            #print(l1)
            if token and not is_nan(token):
                #print('l1:', l1)
                #print('tag:', tag)
                #print('B-'+tag in l1)
                if 'B-'+tag in l1:
                    if isEnt==1: # acabou uma e começou outra do mesmo tipo
                        isEnt = 0
                        listaEntidades.append([nomeEntidade,indices,tag])
                        #print('fim, nomeEntidade:', nomeEntidade)
                        nomeEntidade=''
                        indices=[]
                    nomeEntidade = token
                    #print('inicio, nomeEntidade:', nomeEntidade)
                    isEnt = 1
                    indices.append(j)
                    #print(nomeEntidade)
                elif 'I-'+tag in l1:
                    nomeEntidade = nomeEntidade +' '+ token
                    #print('continua, nomeEntidade:', nomeEntidade)
                    indices.append(j)
                else:
                    #print('isDNA:', isDNA)
                    if isEnt==1:
                        isEnt = 0
                        #print('DNA:', nomeEntidade)
                        listaEntidades.append([nomeEntidade,indices,tag])
                        #print('fim, nomeEntidade:', nomeEntidade)
                        nomeEntidade=''
                        indices=[]   
                j=j+1
            else:
                frase=dicEntidades[i][0]
                entidadesTava=dicEntidades[i][1]
                #print(listaEntidades)
                #print(frase)
                if len(listaEntidades)>0:
                    for elemento in listaEntidades:
                        entidadesTava.append(elemento)
                dicEntidades[i] = [frase, entidadesTava]
                listaEntidades=[]
                j=0
                i=i+1    
            #if i>10:
            #    break

        i=0
        j=0    
        for token, l1 in zip(tokens, label2):
            #print(token)
            #print(l1)
            if token and not is_nan(token):
                #print('l1:', l1)
                #print('tag:', tag)
                #print('B-'+tag in l1)
                if 'B-'+tag in l1:
                    if isEnt==1: # acabou uma e começou outra do mesmo tipo
                        isEnt = 0
                        listaEntidades.append([nomeEntidade,indices,tag])
                        #print('fim, nomeEntidade:', nomeEntidade)
                        nomeEntidade=''
                        indices=[]
                    nomeEntidade = token
                    #print('inicio, nomeEntidade:', nomeEntidade)
                    isEnt = 1
                    indices.append(j)
                    #print(nomeEntidade)
                elif 'I-'+tag in l1:
                    nomeEntidade = nomeEntidade +' '+ token
                    #print('continua, nomeEntidade:', nomeEntidade)
                    indices.append(j)
                else:
                    #print('isDNA:', isDNA)
                    if isEnt==1:
                        isEnt = 0
                        #print('DNA:', nomeEntidade)
                        listaEntidades.append([nomeEntidade,indices,tag])
                        #print('fim, nomeEntidade:', nomeEntidade)
                        nomeEntidade=''
                        indices=[]   
                j=j+1
            else:
                frase=dicEntidades[i][0]
                entidadesTava=dicEntidades[i][1]
                if len(listaEntidades)>0:
                    for elemento in listaEntidades:
                        entidadesTava.append(elemento)
                dicEntidades[i] = [frase, entidadesTava]
                listaEntidades=[]
                j=0
                i=i+1    
            #if i>10:
            #    break        
    return dicEntidades
    
#for token, l1, l2, l3, l4 in zip(tokens, label1, label2, label3, label4):
#    print(token, l1, l2, l3, l4)
#    break
dicEntidadesTeste = getEntidades(arquivos)
dicEntidadesTeste[10]

        token Label1 Label2 Label3 Label4
0       There      O      O      O      O
1          is      O      O      O      O
2           a      O      O      O      O
3      single      O      O      O      O
4  methionine  B-DNA      O      O      O


[' Employing the EBV - transformed human B cell line SKW6 . 4 , we demonstrate that platelet - activating factor significantly alters membrane phospholipid metabolism indicated by the incorporation of 32P into phosphatidylcholine , phosphatidylinositol , and phosphatidic acid but not significantly into phosphatidylethanolamine at concentrations ranging from 10 ( - 9 ) to 10 ( - 6 ) M .',
 [['human B cell line', [5, 6, 7, 8], 'cell_line'],
  ['SKW6 . 4', [9, 10, 11], 'cell_line'],
  ['EBV - transformed human B cell line', [2, 3, 4, 5, 6, 7, 8], 'cell_line'],
  ['platelet - activating factor', [16, 17, 18, 19], 'protein']]]

In [5]:
len(dicEntidadesTeste)

1854

In [6]:
dicEntidadesTeste[92]

[' The functional domains of the murine Thy - 1 gene promoter .',
 [['murine Thy - 1 gene promoter', [5, 6, 7, 8, 9, 10], 'DNA']]]

In [7]:
dicEntidadesTeste[93]

[" The Thy - 1 gene promoter resembles a ' housekeeping ' promoter in that it is located within a methylation - free island , lacks a canonical TATA box , and displays heterogeneity in the 5 ' - end termini of the mRNA .",
 [['Thy - 1 gene promoter', [1, 2, 3, 4, 5], 'DNA'],
  ["' housekeeping ' promoter", [8, 9, 10, 11], 'DNA'],
  ['methylation - free island', [19, 20, 21, 22], 'DNA'],
  ['canonical TATA box', [26, 27, 28], 'DNA'],
  ["5 ' - end termini", [35, 36, 37, 38, 39], 'RNA'],
  ['mRNA', [42], 'RNA']]]

In [8]:
save_obj('dicEntidadesTesteGenia', dicEntidadesTeste)

In [9]:
dicEntidadesTeste[1]

[' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
 [['homeodomain', [2], 'DNA'],
  ['HB24', [4], 'DNA'],
  ['homeodomain', [32], 'DNA'],
  ['Drosophila homeodomain', [31, 32], 'DNA']]]

In [10]:
arquivos = pd.read_csv('CRAM/genia.train.iob2', delimiter='\t', skip_blank_lines = False, header = None)
arquivos.columns=["token", "Label1", "Label2", "Label3", "Label4"]
dicEntidadesTrain = getEntidades(arquivos)
save_obj('dicEntidadesTrainGenia', dicEntidadesTrain)

In [11]:
arquivos = pd.read_csv('CRAM/genia.dev.iob2', delimiter='\t', skip_blank_lines = False, header = None)
arquivos.columns=["token", "Label1", "Label2", "Label3", "Label4"]
dicEntidadesDev = getEntidades(arquivos)
save_obj('dicEntidadesDevGenia', dicEntidadesDev)